<a href="https://colab.research.google.com/github/BeBraveBeCurious/BWM/blob/master/Choquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import itertools
from cvxopt import solvers, matrix

In [2]:
class ChoquetIntegral:

    def __init__(self):
        """Instantiation of a ChoquetIntegral.
           This sets up the ChI. It doesn't take any input parameters
           because you may want to use pass your own values in(as opposed
           to learning from data). To instatiate, use
           chi = ChoquetIntegral.ChoquetIntegral()
        """
        self.trainSamples, self.trainLabels = [], []
        self.testSamples, self.testLabels = [], []
        self.N, self.numberConstraints, self.M = 0, 0, 0
        self.g = 0
        self.fm = []
        self.type = []


    def train_chi(self, x1, l1):
        """
        This trains this instance of your ChoquetIntegral w.r.t x1 and l1.
        :param x1: These are the training samples of size N x M(inputs x number of samples)
        :param l1: These are the training labels of size 1 x M(label per sample)
        """
        self.type = 'quad'
        self.trainSamples = x1
        self.trainLabels = l1
        self.N = self.trainSamples.shape[0]
        self.M = self.trainSamples.shape[1]
        print("Number Inputs : ", self.N, "; Number Samples : ", self.M)
        self.fm = self.produce_lattice()

        return self



    def chi_quad(self, x2):
        """
        This will produce an output for this instance of the ChI
        This will use the learned(or specified) Choquet integral to
        produce an output w.r.t. to the new input.
        :param x2: testing sample
        :return: output of the choquet integral.
        """
        if self.type == 'quad':
            n = len(x2)
            pi_i = np.argsort(x2)[::-1][:n] + 1
            ch = x2[pi_i[0] - 1] * (self.fm[str(pi_i[:1])])
            for i in range(1, n):
                latt_pti = np.sort(pi_i[:i + 1])
                latt_ptimin1 = np.sort(pi_i[:i])
                ch = ch + x2[pi_i[i] - 1] * (self.fm[str(latt_pti)] - self.fm[str(latt_ptimin1)])
            return ch
        else:
            print("If using sugeno measure, you need to use chi_sugeno.")


    def produce_lattice(self):
        """
            This method builds is where the lattice(or FM variables) will be learned.
            The FM values can be found via a quadratic program, which is used here
            after setting up constraint matrices. Refer to papers for complete overview.
        :return: Lattice, the learned FM variables.
        """

        fm_len = 2 ** self.N - 1  # nc
        E = np.zeros((fm_len, fm_len))  # D
        L = np.zeros(fm_len)  # f
        index_keys = self.get_keys_index()
        for i in range(0, self.M):  # it's going through one sample at a time.
            l = self.trainLabels[i]  # this is the labels
            fm_coeff = self.get_fm_class_img_coeff(index_keys, self.trainSamples[:, i], fm_len)  # this is Hdiff
            # print(fm_coeff)
            L = L + (-2) * l * fm_coeff
            E = E + np.matmul(fm_coeff.reshape((fm_len, 1)), fm_coeff.reshape((1, fm_len)))

        G, h, A, b = self.build_constraint_matrices(index_keys, fm_len)
        sol = solvers.qp(matrix(2 * E, tc='d'), matrix(L.T, tc='d'), matrix(G, tc='d'), matrix(h, tc='d'),
                         matrix(A, tc='d'), matrix(b, tc='d'))

        g = sol['x']
        Lattice = {}
        for key in index_keys.keys():
            Lattice[key] = g[index_keys[key]]
        return Lattice


    def build_constraint_matrices(self, index_keys, fm_len):
        """
        This method builds the necessary constraint matrices.
        :param index_keys: map to reference lattice components
        :param fm_len: length of the fuzzy measure
        :return: the constraint matrices
        """

        vls = np.arange(1, self.N + 1)
        line = np.zeros(fm_len)
        G = line
        line[index_keys[str(np.array([1]))]] = -1.
        h = np.array([0])
        for i in range(2, self.N + 1):
            line = np.zeros(fm_len)
            line[index_keys[str(np.array([i]))]] = -1.
            G = np.vstack((G, line))
            h = np.vstack((h, np.array([0])))
        for i in range(2, self.N + 1):
            parent = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in parent:
                for j in range(len(latt_pt) - 1, len(latt_pt)):
                    children = np.array(list(itertools.combinations(latt_pt, j)))
                    for latt_ch in children:
                        line = np.zeros(fm_len)
                        line[index_keys[str(latt_ch)]] = 1.
                        line[index_keys[str(latt_pt)]] = -1.
                        G = np.vstack((G, line))
                        h = np.vstack((h, np.array([0])))

        line = np.zeros(fm_len)
        line[index_keys[str(vls)]] = 1.
        G = np.vstack((G, line))
        h = np.vstack((h, np.array([1])))

        # equality constraints
        A = np.zeros((1, fm_len))
        A[0, -1] = 1
        b = np.array([1]);

        return G, h, A, b


    def get_fm_class_img_coeff(self, Lattice, h, fm_len):  # Lattice is FM_name_and_index, h is the samples, fm_len
        """
        This creates a FM map with the name as the key and the index as the value
        :param Lattice: dictionary with FM
        :param h: sample
        :param fm_len: fm length
        :return: the fm_coeff
        """

        n = len(h)  # len(h) is the number of the samples
        fm_coeff = np.zeros(fm_len)
        pi_i = np.argsort(h)[::-1][:n] + 1
        for i in range(1, n):
            fm_coeff[Lattice[str(np.sort(pi_i[:i]))]] = h[pi_i[i - 1] - 1] - h[pi_i[i] - 1]
        fm_coeff[Lattice[str(np.sort(pi_i[:n]))]] = h[pi_i[n - 1] - 1]
        np.matmul(fm_coeff, np.transpose(fm_coeff))
        return fm_coeff


    def get_keys_index(self):
        """
        Sets up a dictionary for referencing FM.
        :return: The keys to the dictionary
        """

        vls = np.arange(1, self.N + 1)
        count = 0
        Lattice = {}
        for i in range(0, self.N):
            Lattice[str(np.array([vls[i]]))] = count
            count = count + 1
        for i in range(2, self.N + 1):
            A = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in A:
                Lattice[str(latt_pt)] = count
                count = count + 1
        return Lattice

In [3]:
if __name__ == '__main__':
    # instatiate ChoquetIntegral object
    
    chi = ChoquetIntegral()
    
    # create data samples and labels to produce a max aggregation operation
    
    data = np.random.rand(3, 25)
    labels = np.amax(data, 0)
    
    # train the chi via quadratic program 
    chi.train_chi(data, labels)

    # print out the learned chi variables. (in this case, all 1's) 
    print(chi.fm)

Number Inputs :  3 ; Number Samples :  25
     pcost       dcost       gap    pres   dres
 0: -1.5517e+01 -1.9933e+01  2e+01  4e+00  4e-01
 1: -1.5136e+01 -1.7924e+01  3e+00  4e-02  4e-03
 2: -1.5484e+01 -1.5952e+01  5e-01  4e-03  5e-04
 3: -1.5644e+01 -1.5723e+01  8e-02  4e-05  5e-06
 4: -1.5673e+01 -1.5683e+01  1e-02  4e-07  5e-08
 5: -1.5677e+01 -1.5679e+01  2e-03  4e-09  5e-10
 6: -1.5678e+01 -1.5678e+01  2e-04  4e-11  5e-12
 7: -1.5678e+01 -1.5678e+01  4e-05  4e-13  3e-13
 8: -1.5678e+01 -1.5678e+01  5e-06  4e-15  4e-16
Optimal solution found.
{'[1]': 0.9991797815289991, '[2]': 0.9982591704912399, '[3]': 0.999216636829281, '[1 2]': 0.9998204244476965, '[1 3]': 0.9998935543130755, '[2 3]': 0.9998437953095702, '[1 2 3]': 1.0}


In [4]:
print(data)

[[0.56903708 0.6658986  0.99876895 0.42518015 0.09485378 0.66675754
  0.0529299  0.97070373 0.619074   0.22022572 0.54879446 0.89146485
  0.74377095 0.08428956 0.53506016 0.74906528 0.84662447 0.27370062
  0.57693272 0.11094072 0.88223277 0.33895508 0.13635191 0.29297978
  0.47294588]
 [0.94619876 0.27106214 0.4215942  0.53602007 0.48518513 0.10297508
  0.24900413 0.57524246 0.04021041 0.50566753 0.20404259 0.06638178
  0.11740658 0.23435837 0.65277491 0.16463801 0.69173158 0.19129738
  0.90090434 0.09484511 0.82925084 0.41629096 0.5638254  0.98090415
  0.75854004]
 [0.94901086 0.87599776 0.34557888 0.09295722 0.16944146 0.78450297
  0.90209288 0.89937107 0.19960305 0.53091742 0.72393962 0.07756487
  0.87576868 0.33201924 0.81927858 0.73517332 0.5904834  0.13742578
  0.08766599 0.76174095 0.40211365 0.68238279 0.95646999 0.97513895
  0.84854252]]


In [5]:
print(labels)

[0.94901086 0.87599776 0.99876895 0.53602007 0.48518513 0.78450297
 0.90209288 0.97070373 0.619074   0.53091742 0.72393962 0.89146485
 0.87576868 0.33201924 0.81927858 0.74906528 0.84662447 0.27370062
 0.90090434 0.76174095 0.88223277 0.68238279 0.95646999 0.98090415
 0.84854252]
